### Problem sentence


You have to create two separate functions that are not allowed to comunicate between them i.e. can't use global variables.
I generate an array of size "array_size" with random integers with values less than "max_int" (values can repeat).
This array is passed to the first function that is tasked to arrange the elements in the array in whatever manner it sees fit and returns the newly ordered array to me.
I take a random element out of the array and I pass it to the second function.
The second function is tasked with finding what element I removed from the array.

Please fill in the two functions with your code.

Good luck! :)


### Import Libraries


In [266]:
import numpy as np

In [267]:
max_int: int = 10000
array_size: int = 100000

### Generate array of random numbers


In [268]:
array: np.ndarray = np.random.randint(0, max_int, size=array_size)

### Run through first program


In [269]:
from numpy.random import default_rng


def function_1(array: np.ndarray) -> np.ndarray:
    cpy_array = np.ascontiguousarray(array, dtype=np.int64)
    cpy_array.sort()

    rng = default_rng(cpy_array.sum(dtype=np.int64))
    rng.shuffle(cpy_array)

    return cpy_array

In [270]:
array_1 = function_1(array)

### Remove a number from the list


In [271]:
np.random.seed()
idx = np.random.randint(0, len(array_1))
removed_element = int(array_1[idx])
array_1_modified = np.delete(array_1, idx)

### Run through second program


In [272]:
from joblib import Parallel, delayed
from numpy.random import default_rng
import numba


@numba.njit(fastmath=True)
def compare_with_missing(arr_shuffled, arr):
    n = arr.shape[0]
    m = arr_shuffled.shape[0]
    mismatch_count = 0
    i, j = 0, 0

    while i < n and j < m:
        if arr_shuffled[j] == arr[i]:
            i += 1
            j += 1
        else:
            mismatch_count += 1
            j += 1
            if mismatch_count > 1:
                return False

    mismatch_count += m - j
    return mismatch_count == 1


def _process_chunk(array: np.ndarray, array_sum: int, start_i: int, end_i: int) -> int:
    n = array.size
    cpy_array = np.empty(n + 1, dtype=np.int64)

    for i in range(start_i, end_i):
        cpy_array[:n] = array
        cpy_array[n] = i

        cpy_array.sort()

        rng = default_rng(array_sum + i)
        rng.shuffle(cpy_array)

        if compare_with_missing(cpy_array, array):
            return i
    return -1


def function_2(
    array: np.ndarray, max_int: int, chunk_size: int = 100, n_jobs: int = -1
) -> int:
    array = np.ascontiguousarray(array, dtype=np.int64)
    array_sum = array.sum(dtype=np.int64)

    n_chunks = (max_int + chunk_size - 1) // chunk_size

    tasks = (
        delayed(_process_chunk)(
            array, array_sum, i * chunk_size, min((i + 1) * chunk_size, max_int)
        )
        for i in range(n_chunks)
    )

    results = Parallel(n_jobs=n_jobs)(tasks)

    valid_is = [res for res in results if res != -1]

    # Return the smallest valid i or -1 if none
    return valid_is[0] if valid_is else -1

In [273]:
answer = function_2(array_1_modified, max_int)

In [274]:
answer, removed_element

(9051, 9051)